# Fatigue Detection Using Facial Landmarks and ANN

## Input : coordinates (x,y) of face landmarks 
## Output : Fatigue level


In [1]:
import tensorflow as tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
video_name = "DESFAM_Semaine 2-Vendredi_PVT_H64"
video_infos_path = "data/stage_data_out/videos_infos.csv"
video_path = "data/stage_data_out/DESFAM_Semaine 2-Vendredi_PVT_H64_hog.csv"

In [3]:
df_video_infos = pd.read_csv(video_infos_path)
fps = list(df_video_infos[df_video_infos["video_name"]==video_name]["fps"])[0]
num_min = 5
num_sec = num_min*60
num_frame_by_num_min = int(fps*num_sec)


In [4]:
df_landmarks = pd.read_csv(video_path)
frame_df = df_landmarks["frame"]
df_label = pd.DataFrame(frame_df)
df_label = df_label.append( pd.DataFrame(columns=['Fatigue','Normale']))

df_label.loc[lambda df_label: df_label["frame"] <= num_frame_by_num_min,"Normale"] = 1
df_label.loc[lambda df_label: df_label["frame"] <= num_frame_by_num_min,"Fatigue"] = 0

df_label.loc[lambda df_label: df_label["frame"] > num_frame_by_num_min,"Normale"] = 0
df_label.loc[lambda df_label: df_label["frame"] > num_frame_by_num_min,"Fatigue"] = 1

df_label

,frame,Fatigue,Normale
0,0.0,0,1
1,1.0,0,1
2,2.0,0,1
3,3.0,0,1
4,4.0,0,1
...,...,...,...
12904,30431.0,1,0
12905,30432.0,1,0
12906,30433.0,1,0
12907,30434.0,1,0


In [61]:
mean_landmarks_normale = pd.DataFrame(df_landmarks[df_landmarks["frame"] <= num_frame_by_num_min].mean())
mean_landmarks_fatigue = pd.DataFrame(df_landmarks[df_landmarks["frame"] > num_frame_by_num_min].mean())
mean_landmarks_fatigue = mean_landmarks_fatigue.drop("frame")
mean_landmarks_normale = mean_landmarks_normale.drop("frame")
mean_landmarks_fatigue = mean_landmarks_fatigue.rename(columns={0 : "Fatigue Mean"})
mean_landmarks_normale = mean_landmarks_normale.rename(columns={0 : "Normale Mean"})


In [63]:
mean_landamrks = pd.concat([mean_landmarks_fatigue, mean_landmarks_normale], axis=1)
mean_landamrks["Variance"] = mean_landamrks.var(axis=1)
mean_landamrks["Mean Difference"] = mean_landamrks["Fatigue Mean"] - mean_landamrks["Normale Mean"]
mean_landamrks

,Fatigue Mean,Normale Mean,Variance,Mean Difference
landmarks_1_x,948.928432,940.268943,37.493377,8.659489
landmarks_1_y,458.070312,458.915694,0.357335,-0.845382
landmarks_2_x,949.041713,941.249086,30.362523,7.792628
landmarks_2_y,478.479771,479.452186,0.472795,-0.972415
landmarks_3_x,950.495117,943.606689,23.725223,6.888428
...,...,...,...,...
landmarks_66_y,536.205078,532.847936,5.635202,3.357142
landmarks_67_x,1024.118025,1020.762237,5.630657,3.355788
landmarks_67_y,536.616490,533.585438,4.593638,3.031052
landmarks_68_x,1017.119141,1013.797074,5.518064,3.322067
